<a href="https://colab.research.google.com/github/carlibeisel/Drains_Lower_Boise_River/blob/main/05_compile_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modified by Carli Beisel

Adapted from code written by Bridget Bittmann (2023, Github: bridgetmarie24)

Date originally created: July 21, 2022

Date modified: May 13, 2024

Purpose: Merges flow, climate, and land use annual stats together.

*** cannot use direct output from this in R. It merges Mason Creek and Mason Drain wrong. This was manually corrected.

In [1]:
## --------------- ##
## IMPORT PACKAGES ##
## --------------- ##

import pandas as pd # to work with dataframe
import os # for file paths
import glob # read in a folder of csv
import numpy as np # basic statistics

#connect to Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [36]:
## ------------------------------ ##
## IMPORT ALL DATASETS ##
## ------------------------------ ##

drains = pd.read_csv('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/drainage_flows_out/annual_flow_vals.csv')

land_files = sorted(glob.glob('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/final_metrics/*.csv'))
land = []
for i in land_files:
  land.append(pd.read_csv(i))
land = pd.concat(land)

clim_files = sorted(glob.glob('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/final/*.csv'))
clim = []
for i in clim_files:
  clim.append(pd.read_csv(i))
clim = pd.concat(clim)

hydromet = pd.read_csv('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_output/hydromet_data_out/model_input_hydromet.csv')

ubrb_prcp = pd.read_csv('/content/gdrive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp.csv')
ubrb_prcp = ubrb_prcp.rename(columns={'year': 'Year'}) #rename to fit column structure
ubrb_prcp = ubrb_prcp.rename(columns = {'precip':'ubrb_prcp'}) #rename to fit column structure

pivot_change = pd.read_csv('/content/gdrive/MyDrive/Data/Model Modifications/irrigation_change/irrigation_model_input.csv')
merge_final = merge_final.rename(columns = {'Predicted_Proportion': 'pivot_prop'}) #rename to fit column structure


In [37]:
## ------------------------------ ##
## Use a dictionary to help align ##
## ------------------------------ ##

relate = pd.read_csv('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_input/drainage_flows/dain_relates.csv')
spatial = relate.dropna(subset=['Spatial Name'])
newnames = dict(zip(spatial['Spatial Name'], spatial['NewName']))

clim['NewName'] = clim['NAME'].map(newnames)
clim = clim.drop(['Unnamed: 0', 'NAME'], axis=1)
land['NewName2'] = land['DrainName'].map(newnames)
land = land.drop(['Unnamed: 0', 'DrainName'], axis=1)

In [39]:
## ------------------------------ ##
## Merge the files together ##
## ------------------------------ ##

merge_clim = drains.merge(clim, left_on = ['Year', 'Name'], right_on = ['Year', 'NewName'])
merge_land = merge_clim.merge(land, left_on = ['Year', 'Name'], right_on = ['dates', 'NewName2'])
merge_land = merge_land.drop(['Unnamed: 0', 'NewName', 'dates',  'NewName2'], axis=1)


In [28]:
ubrb_prcp.columns

Index(['year', 'precip'], dtype='object')

In [40]:
## ---------------------------------- ##
## Merge with Model Modificaiton Data ##
## ---------------------------------- ##

merge_pivot_change = merge_land.merge(
    pivot_change,
    on=['Name', 'Year'],
    how='inner'
)
merge_hydromet = merge_land.merge(hydromet, on = 'Year', how='inner')
merge_ubrb_prcp = merge_hydromet.merge(ubrb_prcp, on = 'Year', how='inner')
merge_final = merge_ubrb_prcp

In [43]:
merge_final.columns

Index(['Name', 'Year', 'Sum_AF', 'SD_AF', 'ant_prcp', 'wy_prcp', 'irrig_temp',
       'JuneAug_temp', 'et', 'class1_urban', 'class2_crops', 'contagion',
       'largest_patch_index', 'LP_inflows', 'Max_Fill', 'Carryover',
       'ubrb_prcp'],
      dtype='object')

In [44]:
## ------------------------------ ##
##        Export the file         ##
## ------------------------------ ##
merge_final.to_csv('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/compile_data_out/model_input.csv')